In [2]:
from selenium import webdriver
# 웹 브라우저 구동을 위한 셀레늄 패키지의 웹드라이버 모듈을 임포트 합니다
from bs4 import BeautifulSoup
# 웹 문서에서 원하는 정보를 추출하기 위한 BeautifulSoup 모듈도 가져옵니다
import urllib.parse
# 가져온 이미지를 저장하기 위한 urllib 패키지도 가져옵니다
from selenium.common.exceptions import ElementNotVisibleException
# 구글 이미지 검색을 가져오기 위해 나중에 쓸 예외처리를 위해 오류또한 임포트해 놓습니다
import time
# 이미지 검색이기 때문에 로딩시간이 있어 페이지의 로딩을 기다려 줄 time 패키지도 가져와 놓습니다.

In [3]:

keyword = urllib.parse.quote('보노보노')
# 우선 keyword라는 변수에 저희가 검색할 보노보노를 넣어놓습니다. 
driverpath = '/Users/joono/chromedriver'
# 셀레늄은 웹드라이버의 주소를 인식해야 구동이 가능하기 때문에 다운받아놓은 크롬드라이버의 경로를 지정해 놓습니다.
url = 'https://www.google.co.kr/search?q=' + keyword + '&source=lnms&tbm=isch&sa=X&ved=0ahUKEwi9mIGurP_ZAhVFnZQKHTCmDbgQ_AUICigB&biw=709&bih=780&dpr=2'
# 만든 이미지 검색 주소를 삽입합니다.

In [5]:
dr = webdriver.Chrome(driverpath)
# dr 이라는 변수에 저희가 사용할 웹드라이버를 불러옵니다.
dr.get(url)
# get을 사용해 저장해 놓았던 url변수의 주소를 가진 사이트를 오픈합니다.
q = True
# while 문을 사용하기 위한 변수 q에 우선 True를 대입해 놓습니다.
while q ==True:
    newHeight = dr.execute_script("return document.body.scrollHeight")
    # newHeight라는 변수에 우선은 현재 브라우저 창의 길이를 가져와 놓습니다. 맨 마지막에 더이상 로딩이 되는지를 
    # 비교하기 위해 가져온 변수입니다.
    dr.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # 그러고 난 후 스크롤을 맨 밑( 바디의 길이만큼) 까지 내려줍니다.
    try:
        dr.find_element_by_xpath('//*[@id="smb"]').click()
        # 예외처리문을 사용해 결과 더보기라는 버튼이 있는지 찾아줍니다. 
    except ElementNotVisibleException:
        # 만약 아직 결과 더보기라는 버튼이 나오지 않았다면, 다시한번 스크롤을 내려줍니다.
        dr.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
    if newHeight == dr.execute_script("return document.body.scrollHeight"):
        # 만약에 내려진 길이와 위에서 저장했던 브라우저 바디의 길이가 같다면, 스크롤을 해도 더이상 로딩이 되지 
        # 않음을 알수 있습니다.
        q = False
        # 그러면 q에 False를 할당하여 while문을 멈춰주도록 합니다. 


In [6]:
# 현재 dr에는 보노보노를 검색한 모든 사진들이 들어가 있습니다.
#여기서 모든 사진을 가져오는 작업을 할텐데, 사진을 저장하려면 사진의 주소인 src가 필요합니다.

drt = dr.page_source
#우선 현재 브라우저의 페이지 소스를 가져옵니다.
sp = BeautifulSoup(drt,'html.parser')
# soup를 사용하여 파싱을 진행해주고
images = sp.find_all('img',attrs = {'class' : 'rg_ic rg_i'})
# 이미지의 주소를 사용해 모든 이미지를 images에 저장합니다. 위의 attribute는 어떻게 알수 있는지 봅시다.
# 여러가지 이미지를 검사해본 결과 class 이름은 바뀌지 않는다는 것을 볼 수 있습니다. 따라서 rg_ic rg_i라는 
# class를 가진 이미지를 가져오는데 성공합니다. 


In [7]:
srcs = []
#src 를 저장할 리스트를 하나 만듭니다.
for image in images:
    srcs.append(image.get('src'))
# 저장했던 images에서 각각의 image를 뽑아 src를 가져온 후, srcs라는 리스트에 하나하나 저장해 줍니다.

In [8]:
import sys
# 저장경로 설정을 위해 sys를 불러옵니다.
for idx,image_src in enumerate(srcs):
    #이미지의 파일명을 지정하기 위해 enumerate를 사용해 index까지 불러옵니다.
    if image_src != None:
        urllib.request.urlretrieve(str(image_src),sys.path[1]+'/files/image'+str(idx+1)+'.jpg')
        # 제 프로젝트 안에있는 files라는 폴더에 image + 파일의 인덱스 순서를 추가하여 .jpg파일을 만듭니다. 